#### config.py

In [1]:
%%writefile config.py
import os
import torch 
BATCH_SIZE = 32
RESIZE_TO = 224
NUM_EPOCHS = 100
NUM_WORKER = os.cpu_count()
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TRAIN_DIR = [
    #'/kaggle/input/vn-celeb/VN-celeb/1 percent/train',
    #'/kaggle/input/vn-celeb/VN-celeb/full/train',
    '/kaggle/input/celeb-faces/full/train',
]

VAL_DIR = [
    #'/kaggle/input/vn-celeb/VN-celeb/1 percent/val',
     #'/kaggle/input/vn-celeb/VN-celeb/full/val',
    '/kaggle/input/celeb-faces/full/val',
]

Writing config.py


#### dataset.py

In [2]:
%%writefile dataset.py
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from PIL import Image
import os
import glob
from config import RESIZE_TO
from tqdm.auto import tqdm
import random

class FaceDataset(Dataset):
    def __init__(self, root_dir, transforms = None) -> None:
        super().__init__()
        self.img_paths = glob.glob(os.path.join(root_dir, '*/*.jpg')) + glob.glob(os.path.join(root_dir, '*/*.png'))
        self.labels = [os.path.basename(os.path.abspath(os.path.join(path, os.pardir))) for path in self.img_paths]
        self.transforms = transforms

    def __getitem__(self, index):
        img_path, label = self.img_paths[index], self.labels[index]
        img = Image.open(img_path).convert('RGB')
        if self.transforms is not None:
            img = self.transforms(img)
        return img, label
    
    def __len__(self):
        return len(self.img_paths)
    


class ValidationDataset(Dataset):
    def __init__(self, root_dir, transforms = None, max_files=10):
        self.transforms = transforms
        self.triplets = []
        folders = os.listdir(root_dir)

        for folder in tqdm(folders):
            path = os.path.join(root_dir, folder)
            files = list(os.listdir(path))[:max_files]
            num_files = len(files)

            for i in range(num_files-1):
                for j in range(i+1, num_files):
                    anchor = os.path.join(root_dir, folder, files[i])
                    positive = os.path.join(root_dir, folder, files[j])

                    neg_folder = folder
                    while neg_folder == folder:
                        neg_folder = random.choice(folders)
                    neg_file = random.choice( os.listdir(os.path.join(root_dir, neg_folder)))
                    negative = os.path.join(root_dir, neg_folder, neg_file)
                    self.triplets.append([anchor, positive, negative])
        
    def __getitem__(self, index):
        anchor, positive, negative = self.triplets[index]
        anchor, positive, negative = Image.open(anchor).convert('RGB'), Image.open(positive).convert('RGB'), Image.open(negative).convert('RGB')
        if self.transforms is not None:
            anchor, positive, negative = self.transforms(anchor), self.transforms(positive), self.transforms(negative)
        return anchor, positive, negative
    
    def __len__(self):
        return len(self.triplets)  


train_tfms = transforms.Compose([
    transforms.Resize(size=(RESIZE_TO, RESIZE_TO)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(
        brightness=0.25, contrast=0.25, saturation=0.25
    ),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
    

val_tfms = transforms.Compose([
    transforms.Resize(size=(RESIZE_TO, RESIZE_TO)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

if __name__ == '__main__':
    from config import TRAIN_DIR
    import random
    import cv2
    import numpy as np
    trainset = FaceDataset(root_dir=TRAIN_DIR[0], transforms=train_tfms)
    idx = random.randint(0,trainset.__getlen__() -1)
    ori_img = cv2.imread(trainset.img_paths[idx])
    transformed_img, label = trainset.__getitem__(idx)
    transformed_img = transformed_img.permute(1,2,0).numpy()
    transformed_img = cv2.cvtColor((transformed_img*255).astype(np.uint8), cv2.COLOR_RGB2BGR)

    delta_w = 224 - ori_img.shape[1]
    delta_h = 224 - ori_img.shape[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    ori_img = cv2.copyMakeBorder(ori_img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0,0,0))

    img = cv2.hconcat([ori_img, transformed_img])
    cv2.imwrite('./out.png', img)
    print(label)

Writing dataset.py


#### loss.py

In [3]:
%%writefile loss.py
import torch
import torch.nn as nn

def pairwise_distance_torch(embeddings, device):
    """Computes the pairwise distance matrix with numerical stability.
    output[i, j] = || feature[i, :] - feature[j, :] ||_2
    Args:
      embeddings: 2-D Tensor of size [number of data, feature dimension].
    Returns:
      pairwise_distances: 2-D Tensor of size [number of data, number of data].
    """

    # pairwise distance matrix with precise embeddings
    precise_embeddings = embeddings.to(dtype=torch.float32)

    c1 = torch.pow(precise_embeddings, 2).sum(axis=-1)   # (batch_size, 1)
    c2 = torch.pow(precise_embeddings.transpose(0, 1), 2).sum(axis=0) # (1, batch_size)
    c3 = precise_embeddings @ precise_embeddings.transpose(0, 1)

    c1 = c1.reshape((c1.shape[0], 1))    # (batch_size, 1)
    c2 = c2.reshape((1, c2.shape[0]))   # (1, batch_size)
    c12 = c1 + c2
    pairwise_distances_squared = c12 - 2.0 * c3

    # Deal with numerical inaccuracies. Set small negatives to zero.
    pairwise_distances_squared = torch.max(pairwise_distances_squared, torch.tensor([0.]).to(device))
    # Get the mask where the zero distances are at.
    error_mask = pairwise_distances_squared.clone()
    error_mask[error_mask > 0.0] = 1.
    error_mask[error_mask <= 0.0] = 0.

    pairwise_distances = torch.mul(pairwise_distances_squared, error_mask)

    # Explicitly set diagonals to zero.
    mask_offdiagonals = torch.ones((pairwise_distances.shape[0], pairwise_distances.shape[1])) - torch.diag(torch.ones(pairwise_distances.shape[0]))
    pairwise_distances = torch.mul(pairwise_distances.to(device), mask_offdiagonals.to(device))
    return pairwise_distances

def TripletSemiHardLoss(y_true, y_pred, device, margin=1.0):
    """Computes the triplet loss_functions with semi-hard negative mining.
       The loss_functions encourages the positive distances (between a pair of embeddings
       with the same labels) to be smaller than the minimum negative distance
       among which are at least greater than the positive distance plus the
       margin constant (called semi-hard negative) in the mini-batch.
       If no such negative exists, uses the largest negative distance instead.
       See: https://arxiv.org/abs/1503.03832.
       We expect labels `y_true` to be provided as 1-D integer `Tensor` with shape
       [batch_size] of multi-class integer labels. And embeddings `y_pred` must be
       2-D float `Tensor` of l2 normalized embedding vectors.
       Args:
         margin: Float, margin term in the loss_functions definition. Default value is 1.0.
         name: Optional name for the op.
       """

    labels, embeddings = y_true, y_pred

    # Reshape label tensor to [batch_size, 1].
    lshape = labels.shape
    labels = torch.reshape(labels, [lshape[0], 1])

    pdist_matrix = pairwise_distance_torch(embeddings, device)

    # Build pairwise binary adjacency matrix.
    adjacency = torch.eq(labels, labels.transpose(0, 1))
    # Invert so we can select negatives only.
    adjacency_not = adjacency.logical_not()

    batch_size = labels.shape[0]

    # Compute the mask.
    pdist_matrix_tile = pdist_matrix.repeat(batch_size, 1)
    adjacency_not_tile = adjacency_not.repeat(batch_size, 1)

    transpose_reshape = pdist_matrix.transpose(0, 1).reshape(-1, 1)
    greater = pdist_matrix_tile > transpose_reshape

    mask = adjacency_not_tile & greater

    # final mask
    mask_step = mask.to(dtype=torch.float32)
    mask_step = mask_step.sum(axis=1)
    mask_step = mask_step > 0.0
    mask_final = mask_step.reshape(batch_size, batch_size)
    mask_final = mask_final.transpose(0, 1)

    adjacency_not = adjacency_not.to(dtype=torch.float32)
    mask = mask.to(dtype=torch.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    axis_maximums = torch.max(pdist_matrix_tile, dim=1, keepdim=True)
    masked_minimums = torch.min(torch.mul(pdist_matrix_tile - axis_maximums[0], mask), dim=1, keepdim=True)[0] + axis_maximums[0]
    negatives_outside = masked_minimums.reshape([batch_size, batch_size])
    negatives_outside = negatives_outside.transpose(0, 1)

    # negatives_inside: largest D_an.
    axis_minimums = torch.min(pdist_matrix, dim=1, keepdim=True)
    masked_maximums = torch.max(torch.mul(pdist_matrix - axis_minimums[0], adjacency_not), dim=1, keepdim=True)[0] + axis_minimums[0]
    negatives_inside = masked_maximums.repeat(1, batch_size)

    semi_hard_negatives = torch.where(mask_final, negatives_outside, negatives_inside)

    loss_mat = margin + pdist_matrix - semi_hard_negatives

    mask_positives = adjacency.to(dtype=torch.float32) - torch.diag(torch.ones(batch_size)).to(device)
    num_positives = mask_positives.sum()

    triplet_loss = (torch.max(torch.mul(loss_mat, mask_positives), torch.tensor([0.]).to(device))).sum() / num_positives
    triplet_loss = triplet_loss.to(dtype=embeddings.dtype)
    return triplet_loss


class SemiHardTripletLoss(nn.Module):
    def __init__(self, device, margin=1.0):
        super().__init__()
        self.device = device
        self.margin = margin
    def forward(self, input, target, **kwargs):
        return TripletSemiHardLoss(target, input, self.device, self.margin)

Writing loss.py


#### model.py

In [4]:
%%writefile model.py
import torch 
from torch import nn
from torch.nn import functional as F
import torchvision

class MobilenetV2Embedding(nn.Module):
    def __init__(self, out_features=192):
        super(MobilenetV2Embedding, self).__init__()
        net = torchvision.models.mobilenet_v2(pretrained=True)
        net.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=1280, out_features=out_features)
        )
        self.feature_extractor = net
    
    def forward(self, x):
        out = self.feature_extractor(x)
        out = F.normalize(out, p=2, dim=1)
        return out
    
    

class Resnet50Embedding(nn.Module):
    def __init__(self, out_features=192):
        super(Resnet50Embedding, self).__init__()
        net = torchvision.models.resnet50(pretrained=True)
        net.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=1000, out_features=out_features)
        )
        self.feature_extractor = net
    
    def forward(self, x):
        out = self.feature_extractor(x)
        out = F.normalize(out, p=2, dim=1)
        return out

Writing model.py


#### utils.py

In [5]:
%%writefile utils.py
import torch 
import torch.nn.functional as F
from torch.utils.data.sampler import BatchSampler
import numpy as np
from tqdm.auto import tqdm

class BalancedBatchSampler(BatchSampler):
    """
    BatchSampler - from a MNIST-like dataset, samples n_classes and within these classes samples n_samples.
    Returns batches of size n_classes * n_samples
    """
    def __init__(self, dataset, n_classes, n_samples):
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.batch_size = self.n_classes * self.n_samples
        self.dataset = dataset
    
        self.labels = []
        for i in tqdm(range(dataset.__len__())):
            _, label = self.dataset.__getitem__(i)
            self.labels.append(int(label))
        self.labels = torch.LongTensor(self.labels)
        self.labels_set = list(set(self.labels.numpy()))
        self.labels2indices = {label: np.where(self.labels.numpy() == label)[0] for label in self.labels_set}
        
        # Shuffle the labels
        for label in self.labels_set:
            np.random.shuffle(self.labels2indices[label])
        
        self.n_visited_samples = {label: 0 for label in self.labels_set}
        self.count = 0

    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < len(self.dataset):
            classes = np.random.choice(self.labels_set, self.n_classes, replace=False)

            indices = []
            
            for class_ in classes:
                indices.extend(self.labels2indices[class_][
                            self.n_visited_samples[class_]:self.n_visited_samples[class_] + self.n_samples])
                 
                self.n_visited_samples[class_] += self.n_samples
             
                if self.n_visited_samples[class_] + self.n_samples > len(self.labels2indices[class_]):
                    np.random.shuffle(self.labels2indices[class_])
                    self.n_visited_samples[class_] = 0
            yield indices
            self.count += self.n_classes * self.n_samples



     
    def __len__(self):
        return len(self.dataset) // self.batch_size
    

def semi_triplet_hard_mining(embeddings, labels, margin = 1):
    embeddings = embeddings.detach().cpu()
    num = embeddings.shape[0]

    dist = torch.sum(embeddings ** 2, dim=1) + torch.sum(embeddings ** 2, dim=1).view(-1, 1) - 2 * embeddings.matmul(embeddings.T)
    dist = F.relu(dist).sqrt()
    dist = dist.numpy()


    hori = labels.expand((num, num))
    verti = labels.view(-1, 1).expand((num, num))
    mask = (hori == verti).numpy().astype(np.int) # Same label = 1, different = 0
    anchor = []
    posi_index = []
    nega_index = []
    for i in range(dist.shape[0]): 
        for j in range(dist.shape[0]):
          if mask[i,j] == 0.: 
            continue 
          if i == j: 
            continue 
          dp = dist[i,j]
          for k in range(dist.shape[0]):
            if mask[i,k] == 1: 
              continue 
            dn = dist[i,k]
            loss = dp - dn + margin 
            if loss > 0: 
              anchor.append(i)
              posi_index.append(j)
              nega_index.append(k)

    anchor = np.asarray(anchor)
    posi_index = np.asarray(posi_index)
    nega_index = np.asarray(nega_index) 
    batch_semi = np.vstack([anchor, posi_index, nega_index]).T
    return batch_semi


Writing utils.py


#### train.py

In [6]:
%%writefile train.py
import os
import argparse
import torch
from torch.utils.data import DataLoader
import cv2
import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter


from model import MobilenetV2Embedding, Resnet50Embedding
from dataset import FaceDataset, ValidationDataset,  train_tfms, val_tfms
from loss import SemiHardTripletLoss
from utils import BalancedBatchSampler
from config import TRAIN_DIR, VAL_DIR, DEVICE
from PIL import Image

if __name__ == "__main__":
    #TODO: Create ArgumentParser object 
    parser = argparse.ArgumentParser(description="Training script for Face Identification")
    # Add arguments
    parser.add_argument('--lr', type=float, default=0.001 , help='learning rate')
    parser.add_argument('--epochs', type=int, default=100, help='epochs')
    parser.add_argument('--n_classes', type=int, default=25, help='The number of chosen classes')
    parser.add_argument('--n_samples', type=int, default=10, help='the number of chosen samples in each class')
    parser.add_argument('--margin', type=float, default=0.5, help='acceptable margin')
    parser.add_argument('--max_files', type=int, default=5, help='the maximum number of files per folder in validation')
    parser.add_argument('--patience', type=int, default=10, help='early stopping')
    parser.add_argument('--logdir', type=str, default='./exp', help='tensorboard')
    # Parse the command-line arguments
    args = parser.parse_args()

    #TODO: Hyper-parameters initialization
    print('-------------------- Hyper-parameters initialization -----------------------')
    LR, EPOCHS, N_CLASSES, N_SAMPLES, MARGIN, MAX_FILES, PATIENCE = args.lr, args.epochs, args.n_classes, args.n_samples, args.margin, args.max_files, args.patience
    writer = SummaryWriter(log_dir=args.logdir)

    #TODO: dataset initialization
    print('-------------------- Dataset initialization -----------------------')
    trainset = FaceDataset(TRAIN_DIR[0], train_tfms)
    print(f"Number of training samples: {trainset.__len__()}")
    valset = ValidationDataset(VAL_DIR[0], val_tfms, max_files=MAX_FILES)
    print(f"Number of validation samples: {valset.__len__()}")

    train_loader = DataLoader(trainset, batch_sampler=BalancedBatchSampler(trainset, N_CLASSES, N_SAMPLES), num_workers=os.cpu_count())
    val_loader = DataLoader(valset, batch_size=64, num_workers=os.cpu_count())

    #TODO: Model initialization
    model = MobilenetV2Embedding(192)
    model = model.to(DEVICE)

    # Total parameters and trainable parameters
    total_params = sum(p.numel() for p in model.parameters())
    print(f"{total_params:,} total parameters.")
    total_trainable_params = sum(
        p.numel() for p in model.parameters() if p.requires_grad)
    print(f"{total_trainable_params:,} training parameters.")

    #TODO: Optimizer and loss function initialization
    optimizer = torch.optim.Adam(model.parameters(), LR)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
    loss_fn = SemiHardTripletLoss(device=DEVICE, margin=MARGIN)

    best_epoch, best_acc = 0, 0

    #TODO: Training epochs
    for epoch in range(EPOCHS):
        # Train
        model.train()
        total_loss = 0
        pbar = tqdm(train_loader)
        for idx, (images, labels) in enumerate(pbar):
            labels = torch.Tensor([int(v) for v in list(labels)]).to(DEVICE)
            images = images.to(DEVICE)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                embeddings = model(images)
                loss = loss_fn(embeddings, labels)
                loss.backward()
                optimizer.step()
            
            total_loss += loss.item()
            pbar.set_description("Epoch {}. Iteration {}/{} Loss {}".format(
                epoch+1, 
                idx+1, 
                len(train_loader), 
                loss.item()))
        scheduler.step()
        total_loss /= len(train_loader)
        writer.add_scalar('Loss', total_loss,global_step=epoch+1)
        
        # Validation
        model.eval()
        pbar = tqdm(val_loader)
        pos_scores,  neg_scores = [], []
        for idx, (anchors, positives, negatives) in enumerate(pbar):
            with torch.inference_mode():
                anchor_embeddings, positive_embeddings, negative_embeddings = model(anchors.to(DEVICE)),  model(positives.to(DEVICE)),  model(negatives.to(DEVICE))
            positive_distance = (anchor_embeddings - positive_embeddings).pow(2).sum(1).pow(.5).cpu().numpy().tolist()
            negative_distance = (anchor_embeddings - negative_embeddings).pow(2).sum(1).pow(.5).cpu().numpy().tolist()
            pos_scores += positive_distance
            neg_scores += negative_distance
    
        accuracy = np.sum(np.array(pos_scores) < np.array(neg_scores)) / len(pos_scores)
#         ap_mean = np.mean(pos_scores)
#         an_mean = np.mean(neg_scores)
#         ap_stds = np.std(pos_scores)
#         an_stds = np.std(neg_scores)

        # Validation
#         model.eval()
#         accuracy = []
#         with torch.inference_mode():
#             anchors, samples = [], []
#             labels = os.listdir(VAL_DIR[0])
#             for i, label in tqdm(enumerate(labels)):
#                 files = sorted(os.listdir(os.path.join(VAL_DIR[0], label)))
#                 for idx, file in enumerate(files):
#                     image = Image.open(os.path.join(VAL_DIR[0], label, file))
#                     tf_image = val_tfms(image).unsqueeze(dim=0).to(DEVICE)
#             #         print(tf_image.shape)
#                     with torch.inference_mode():
#                         embedding = model(tf_image).squeeze()
#                     if idx < len(files)/4:
#                         anchors.append(dict(idx=i,label=label, embedding = embedding, img_path = os.path.join(VAL_DIR[0], label, file)))
#                     else:
#                         samples.append(dict(idx=i,label=label, embedding = embedding, img_path = os.path.join(VAL_DIR[0], label, file)))

#         anchor_embeddings = torch.stack([v['embedding'] for v in anchors])
#         n_true_samples, n_wrong_samples = 0, 0

#         for sample in tqdm(samples):
#             embedding = sample['embedding']
#             distance = (embedding - anchor_embeddings).pow(2).sum(1).pow(0.5)

#             pred_label = torch.argmin(distance).item()

#             if pred_label == int(sample['idx']):
#                 n_true_samples += 1
#             else:
#                 n_wrong_samples += 1
                
        
        
#         accuracy = n_true_samples/(n_true_samples + n_wrong_samples)
        
        writer.add_scalar('accuracy', accuracy, global_step=epoch+1)
        print("Epoch {}. Accuracy {}".format(
                    epoch+1, 
                    accuracy)
                )
        checkpoint = dict(
            model = model.state_dict(),
            optimizer = optimizer.state_dict()
        )
        torch.save(checkpoint, 'last.pth')
    
        if accuracy > best_acc:
            best_acc = accuracy
            best_epoch = epoch
            torch.save(checkpoint, './best.pth')
        
        if epoch - best_epoch > PATIENCE:
            break
        

Writing train.py


In [7]:
!mkdir exp
!python train.py --margin 0.5 --n_classes 20 --n_samples 5 --patience 5
#  parser.add_argument('--lr', type=float, default=0.001 , help='learning rate')
#     parser.add_argument('--epochs', type=int, default=100, help='epochs')
#     parser.add_argument('--n_classes', type=int, default=25, help='The number of chosen classes')
#     parser.add_argument('--n_samples', type=int, default=10, help='the number of chosen samples in each class')
#     parser.add_argument('--margin', type=float, default=0.5, help='acceptable margin')
#     parser.add_argument('--max_files', type=int, default=5, help='the maximum number of files per folder in validation')
#     parser.add_argument('--patience', type=int, default=10, help='early stopping')
#     # Parse the command-line arguments

2024-02-19 14:25:07.406697: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-19 14:25:07.406803: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered


2024-02-19 14:25:07.547268: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


-------------------- Hyper-parameters initialization -----------------------
-------------------- Dataset initialization -----------------------


Number of training samples: 6709


  0%|                                                   | 0/224 [00:00<?, ?it/s]

...

 89%|██████████████████████████████████████     | 31/35 [00:10<00:00,  4.46it/s]

 91%|███████████████████████████████████████▎   | 32/35 [00:10<00:00,  4.85it/s]

 94%|████████████████████████████████████████▌  | 33/35 [00:10<00:00,  4.92it/s]

 97%|█████████████████████████████████████████▊ | 34/35 [00:10<00:00,  5.34it/s]

100%|███████████████████████████████████████████| 35/35 [00:10<00:00,  3.23it/s]
Epoch 35. Accuracy 0.9011389521640091


In [ ]:
!zip -r /kaggle/working/10percent.zip /kaggle/working
from IPython.display import clear_output

clear_output(wait=False)